<a href="https://colab.research.google.com/github/huggingface/deep-rl-class/blob/main/notebooks/bonus-unit1/bonus-unit1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 奖励单元 1: 让我们训练一只 Huggy 🐶 去叼棍子

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit2/thumbnail.png" alt="Bonus Unit 1Thumbnail">

在本笔记本中，我们将通过**教 Huggy 狗拿起棍子然后直接在浏览器中玩它**来巩固我们在第一个单元中学到的知识

⬇️ 这是**你将在本单元结束时实现的示例。** ⬇️（启动 ▶ 以查看）

In [ ]:
%%html
<video controls autoplay><source src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy.mp4" type="video/mp4"></video>

### 环境 🎮

-  Huggy 狗， [Thomas Simonini](https://twitter.com/ThomasSimonini) 基于 [Puppo The Corgi](https://blog.unity.com/technology/puppo-the-corgi-cuteness-overload-with-the-unity-ml-agents-toolkit)创建的环境

### 使用的库 📚

- [MLAgents (Hugging Face version)](https://github.com/huggingface/ml-agents)

我们一直在努力改进我们的教程，所以**如果你在此 Notebook 中发现一些问题**，请[在 Github Repo 上打开一个 issue](https://github.com/huggingface/deep-rl-class/issues)。

## 本章的目标 🏆


在本章的末尾，你将：

- 了解**用于训练 Huggy 的状态空间、动作空间和奖励函数**。
- **训练你自己的 Huggy** 来拿棍子。
- 能够直接在**浏览器中与训练有素的 Huggy 一起玩**。





## 这本 notebook 来自深度强化学习课程
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration"/>

在这门免费的课程中，你将会：

- 📖 学习深度强化学习的**理论和实践**。
- 🧑‍💻 学习使用一些著名的**深度强化学习库**，比如 Stable Baselines3、RL Baselines3 Zoo、CleanRL 和 Sample Factory 2.0。
- 🤖 在**独特的环境**中训练智能体
更多详情请查看课程大纲 👉 https://simoninithomas.github.io/deep-rl-course

别忘了**<a href="http://eepurl.com/ic5ZUD">报名参加课程</a>** （我们收集你的电子邮件以便在每个单元发布时向你**发送链接并提供有关挑战和更新的信息**）。

保持联系的最佳方式是加入我们的 Discord 服务器与社区和我们交流 👉🏻 https://discord.gg/ydHrjt3WP5

## 前置条件 🏗️

在深入研究该章节之前，你需要：

🔲 📚 **通过第 1 单元了解强化学习的基础**（MC、TD、奖励假设...）

🔲 📚 **通过奖励单元 1 阅读 Huggy 的介绍**


## 设置 GPU 💪
- - 为了**加速智能体的训练，我们将使用 GPU**。为此，请点击 `Runtime > Change Runtime type`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Step 1">

- `Hardware Accelerator > GPU`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Step 2">

## 克隆仓库并安装依赖项🔽

- 我们需要克隆仓库，它**包含允许你将训练有素的代理推送到 Hub 的实验版本。**

In [ ]:
%%capture
# Clone this specific repository (can take 3min)
!git clone https://github.com/huggingface/ml-agents/

In [ ]:
%%capture
# Go inside the repository and install the package (can take 3min)
%cd ml-agents
!pip3 install -e ./ml-agents-envs
!pip3 install -e ./ml-agents

## 下载并将环境压缩文件移动到 `./trained-envs-executables/linux/` 下

- 我们的环境可执行文件位于一个压缩文件中。
- 我们需要下载它并将其放置到`./trained-envs-executables/linux/` 位置下


In [ ]:
!mkdir ./trained-envs-executables
!mkdir ./trained-envs-executables/linux

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1zv3M95ZJTWHUVOWT6ckq_cm98nft8gdF' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1zv3M95ZJTWHUVOWT6ckq_cm98nft8gdF" -O ./trained-envs-executables/linux/Huggy.zip && rm -rf /tmp/cookies.txt

使用 `wget` 从 https://drive.google.com/uc?export=download&id=1zv3M95ZJTWHUVOWT6ckq_cm98nft8gdF 下载文件 Huggy.zip。查看在  GDrive 中完整方案并在此处[下载大文件](https://bcrf.biochem.wisc.edu/2021/02/05/download-google-drive-files-using-wget/)

In [ ]:
%%capture
!unzip -d ./trained-envs-executables/linux/ ./trained-envs-executables/linux/Huggy.zip

确保你的文件可访问

## 让我们回顾一下这个环境是如何工作的

### The State Space: what Huggy "perceives."

 Huggy 不会“看到”他的环境。相反，我们向他提供有关环境的信息：

- 目标（摇杆）位置
- 自己和目标之间的相对位置
- 他腿的方向。

鉴于所有这些信息， Huggy **可以决定接下来要采取什么行动来实现他的目标**。

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy.jpg" alt="Huggy" width="100%">


### 动作空间： Huggy 可以做什么动作
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy-action.jpg" alt="Huggy action" width="100%">

**关节电机驱动抱腿**。这意味着为了达到目标， Huggy 需要**学会正确地旋转每条腿的关节马达，这样他才能移动**。

### 奖励函数

奖励函数的设计是为了让 ** Huggy 实现他的目标**：拿到棍子。

请记住，强化学习的基础之一是*奖励假设*：目标可以描述为**预期累积奖励**的最大化**。

在这里，我们的目标是 Huggy **朝着棍子走去，但不要旋转太多**。因此，我们的奖励函数必须转化为这个目标。

我们的奖励函数：

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/reward.jpg" alt="Huggy reward function" width="100%">

- *定向奖励*：我们**奖励他接近目标**。
- *时间惩罚*：在每个动作中给予的固定时间惩罚，以**迫使他尽快到达棍子**。
- *旋转惩罚*：如果 Huggy 旋转太多并且转得太快**，我们会对 Huggy 进行惩罚。
- *达到目标奖励*：我们奖励 Huggy **达到目标**。

## 检查 Huggy 配置文件

- 在 ML-Agents 中，你将**训练超参数定义到 config.yaml 文件中。**

- 对于本笔记本的范围，我们不打算修改超参数，但如果你想尝试作为实验，你还应该尝试修改其他一些超参数，Unity 提供了非常好的文档，在这里解释了它们中的每一个] （https://github.com/Unity-Technologies/ml-agents/blob/main/docs/Training-Configuration-File.md）。

- **如果你想修改超参数**，在Google Colab notebook中，你可以点击这里打开config.yaml: `/content/ml-agents/config/ppo/Huggy.yaml`

- For instance **if you want to save more models during the training** (for now, we save every 200,000 training timesteps). You need to modify:
  - `checkpoint_interval`: The number of training timesteps collected between each checkpoint.
  - `keep_checkpoints`: The maximum number of model checkpoints to keep. 

=> Just keep in mind that **decreasing the `checkpoint_interval` means more models to upload to the Hub and so a longer uploading time** 
We’re now ready to train our agent 🔥.

## 训练我们的智能体

要训​​练我们的智能体，我们只需要**启动 mlagents-learn 并选择包含环境的可执行文件。**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/mllearn.png" alt="ml learn function" width="100%">

使用 ML 智能体，我们运行一个训练脚本。我们定义了四个参数：

1. `mlagents-learn <config>`：超参数配置文件所在路径。
2. `--env`：环境可执行文件所在的位置。
3. `--run_id`：你要为训练运行 ID 指定的名称。
4. `--no-graphics`：在训练期间不启动可视化。

训练模型并使用 `--resume` 标志在中断的情况下继续训练。

> 第一次使用 `--resume` 会失败，尝试再次运行该块以绕过错误。



训练将需要 30 到 45 分钟，具体取决于你的机器（不要忘记**设置 GPU**），去喝个咖啡☕️放松一下🤗。

In [ ]:
!mlagents-learn ./config/ppo/Huggy.yaml --env=./trained-envs-executables/linux/Huggy/Huggy --run-id="Huggy" --no-graphics

## 将智能体推送到🤗 Hub

- 现在我们已经训练了我们的智能体，我们**准备将其推送到 Hub，以便能够在你的浏览器上与 Huggy 一起玩🔥。**

为了能够与社区共享你的模型，还需要执行三个步骤：

1️⃣（如果尚未完成）创建一个 HF 帐户 ➡ https://huggingface.co/join

2️⃣ 登录，然后，你需要存储来自 Hugging Face 网站的身份验证 token。
- 创建一个新 token（https://huggingface.co/settings/tokens）**具有写入权限**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="Create HF Token">

- 复制 token
- 运行下面的单元格并粘贴 token

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

如果你不想使用 Google Colab 或 Jupyter Notebook，则需要改用此命令：`huggingface-cli login`


然后，我们只需要运行 mlagents-push-to-hf 即可。

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/mlpush.png" alt="ml learn function" width="100%">

我们定义了 4 个参数：

1. `--run-id`：训练运行id的名称。
2. `--local-dir`：代理的保存位置，它是 results/<run_id name="None">，所以在我的例子中是 results/First Training。
3. `--repo-id`：你要创建或更新的 Hugging Face 存储库的名称。总是<your huggingface="None" username="None">/<the repo="None" name="None">
如果回购不存在**它将自动创建**
4. `--commit-message`：由于 HF 存储库是 git 存储库，因此你需要定义提交消息。

In [ ]:
!mlagents-push-to-hf --run-id="HuggyTraining" --local-dir="./results/Huggy" --repo-id="ThomasSimonini/ppo-Huggy" --commit-message="Huggy"

否则，如果一切正常，你应该在过程结束时有这个（但使用不同的 url 😆）：



```
Your model is pushed to the hub. You can view your model here: https://huggingface.co/ThomasSimonini/ppo-Huggy
```

它是你的模型存储库的链接。存储库包含一个模型卡，解释如何使用模型、你的 Tensorboard 日志和你的配置文件。 **很棒的是它是一个 git 存储库，这意味着你可以进行不同的提交、使用新推送更新你的存储库、打开 Pull Requests 等。**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/modelcard.png" alt="ml learn function" width="100%">

但现在最好的是：**能够在线玩 Huggy 👀。**

## 和你的 Huggy 一起玩 🐕

这一步最简单：

- 在浏览器中打开游戏 Huggy ：https://singularite.itch.io/huggy

- 点击玩我的 Huggy 模型

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/load-huggy.jpg" alt="load-huggy" width="100%">

1. 在第 1 步中，选择你的模型存储库，即模型 ID（在我的例子中是 ThomasSimonini/ppo-Huggy）。

2. 在第 2 步中，**选择你要重播的模型**：
  - 我有多个，因为我们每 500000 个时间步保存一个模型。
  - 但因为我想要更新的，所以我选择 `Huggy.onnx`

👉很棒的是**尝试使用不同的模型步骤来查看代理的改进。**


恭喜你完成了这个奖励单元！

你现在可以坐下来享受和你的 Huggy 🐶 一起玩耍的乐趣了。并且不要 ** 忘记通过与你的朋友分享 Huggy 来传播热爱 🤗**。如果你在社交媒体上分享它，**请声明我们@huggingface 和我@simoninithomas**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy-cover.jpeg" alt="Huggy cover" width="100%">


## 保持热爱，奔赴山海🤗